<a href="https://colab.research.google.com/github/Nishimura35/weatherwise-marina-nishimura/blob/main/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
!pip install fetch-my-weather
!pip install hands-on-ai

import os

os.environ['HANDS_ON_AI_SERVER'] = 'http://ollama.serveur.au'
os.environ['HANDS_ON_AI_MODEL'] = 'granite3.2'
os.environ['HANDS_ON_AI_API_KEY'] = input('Enter your API key: ')

# Setup and Configuation
import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip
from fetch_my_weather import get_weather
from hands_on_ai.chat import get_response

# Add any other setup code here


# Weather Data Functions

def get_user_location():
  """ユーザーから場所の入力を受け取る"""
  pass

def get_user_date():
  """ユーザーから日付情報（today/tomorrow/具体的日付）を受け取る"""
  pass

def convert_to_date(date_input):
  """today/tomorrowを実際の日付（YYYY-MM-DD）に変換"""
  pass

def get_weather_data(location, date):
  """fetch-my-weatherを使って天気データを取得"""
  pass

def generate_weather_graph(weather_data):
  """matplotlibを使って天気のグラフを作成"""
  pass

def respond_to_user(graph_image, summary_text):
  """ユーザーに結果を返す（グラフ+テキスト）"""
  pass

# Define get_weather_data() function here
def get_weather_data(location, forecast_days=5):
    """
    Retrieve weather data for a specified location.

    Args:
        location (str): City or location name
        forecast_days (int): Number of days to forecast (1-5)

    Returns:
        dict: Weather data including current conditions and forecast
    """
    pass


# Visualisation Functions
# Define create_temperature_visualisation() and create_precipitation_visualisation() here
def create_temperature_visualisation(weather_data, output_type='display'):
    """
    Create visualisation of temperature data.

    Args:
        weather_data (dict): The processed weather data
        output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

    Returns:
        If output_type is 'figure', returns the matplotlib figure object
        Otherwise, displays the visualisation in the notebook
    """
    pass

# Natural Language Processing
# Define parse_weather_question() and generate_weather_response() here
def parse_weather_question(question):
    """
    Parse a natural language weather question.

    Args:
        question (str): User's weather-related question

    Returns:
        dict: Extracted information including location, time period, and weather attribute
    """
    pass


def generate_weather_response(parsed_question, weather_data):
    """
    Generate a natural language response to a weather question.

    Args:
        parsed_question (dict): Parsed question data
        weather_data (dict): Weather data

    Returns:
        str: Natural language response
    """
    pass


# User Interface


# Main Application Logic
while True:
  location = input("Hi I'm Weatherwise App!\nI can help you check the weather forecast.\nWhich city are you interested in? (type 'exit' to quit)\n")

  if location.lower() == 'exit':
    print("Goodbye! Have a nice day!")
    break

  date = input("Which date do you want to know the weather? (e.g. 2025-05-28)\n")
  if location.lower() == 'exit':
    print("Goodbye! Have a nice day!")
    break

  try:
    weather_data = get_weather_data(location, date)

    if weather_data:
      create_temperature_visualisation(weather_data)
      create_precipitation_visualisation(weather_data)
      break
    else:
      print("Sorry, I couldn't fetch the weather data. Please try again.\n")

  except Exception as e:
    print(f"An error occurred: {e}")
    print("Please try again.\n")





# Testing and Examples
# Include sample input/output for each function